In [49]:
import pandas as pd

df = pd.read_csv('UL-Standards-List.csv')
# drop rows with no description
df = df.dropna(subset=['Description'])
len(df)
# df.head()


2123

In [50]:
# Description to Title Case since there are some descriptions that are all caps
df['Description'] = df['Description'].str.title()





In [51]:

df.iloc[144]['Description']

'Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method) / Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method)'

In [52]:
# Removing duplicates in the description separated by ' / '
def remove_duplicates(string):
    words = string.split(' / ')
    result = []
    for word in words:
        if word not in result:
            result.append(word)
    return ' / '.join(result)  # nothing to join if there are no duplicates

df['Description'] = df['Description'].apply(remove_duplicates)
df.iloc[570]['Description']

'Standard For Category 3 Film-Forming Fluoroprotein (Fffp) Foam Liquid Concentrates'

In [53]:
# Look for non-English descriptions
def isEnglish(s):
    return s.isascii()

df['English?'] = df['Description'].apply(isEnglish)


In [54]:
df['Category'] = ''
df['Topic'] = ''

In [55]:
# Remove non-English descriptions
for idx, row in df.iterrows():
    if row['English?'] == False:
        split_description = row['Description'].split(' / ', maxsplit=1)
        if len(split_description) > 1:
            df.at[idx, 'Topic'] = split_description[0]
        else:
            df.at[idx, 'Topic'] = row['Description']
    else:
        df.at[idx, 'Topic'] = row['Description']



In [56]:
import re

# Split at ';' or ':' or '-'
def split_topic_punc(string):
    split_string = re.split(';|:| - ', row['Topic'], maxsplit=3)
    if len(split_string) > 1:
        return split_string
    else:
        return string

for idx, row in df.iterrows():
    if len(split_topic_punc(row['Topic'])) == 2:
        df.at[idx, 'Category'] = split_topic_punc(row['Topic'])[0].rstrip()
        df.at[idx, 'Topic'] = split_topic_punc(row['Topic'])[1].lstrip()
    elif len(split_topic_punc(row['Topic'])) == 3:
        df.at[idx, 'Category'] = split_topic_punc(row['Topic'])[0].rstrip()
        df.at[idx, 'Topic'] = split_topic_punc(row['Topic'])[1].lstrip() + ' ' + split_topic_punc(row['Topic'])[2].lstrip()
    else:
        df.at[idx, 'Category'] = split_topic_punc(row['Topic'])[0].rstrip()
        df.at[idx, 'Topic'] = split_topic_punc(row['Topic'])[1].lstrip() + ' ' + split_topic_punc(row['Topic'])[2].lstrip() + ' ' + split_topic_punc(row['Topic'])[3].lstrip()



In [57]:
df['Topic'][1505]

'Information Security Requirements For Connected Diabetes Solutions'

In [58]:

def find_words_in_description(string, words):
    start_index = 0
    found_word = ''
    for word in words:
        i = string.find(word, start_index)
        if i != -1:
            start_index = i + len(word)
            found_word = found_word + ' ' + word
        else:
            start_index = start_index
    return (start_index, found_word)

words = ['Standard', 'Method', 'Test', 'Outline', 'Guide', 'Guidance', 'Criteria', 'Procedure', 'Rule', 'Requirement','Enclosure']


for idx, row in df.iterrows():
    i = find_words_in_description(row['Topic'], words)[0]
    final_word = find_words_in_description(row['Topic'], words)[1]
    print(i, final_word)
    if final_word != '':
        df.at[idx, 'Category'] = final_word
        split_topic_desc = re.split(' For | Of ', row['Topic'][i:], maxsplit=1)
        # join the word in description before the split if not in the list of words

        print(split_topic_desc)
        if len(split_topic_desc) > 1:
            lost_words = [word for word in row['Topic'][:i] if word not in words]
            new_string = ''.join(lost_words)
            df.at[idx, 'Category'] = new_string
            df.at[idx, 'Topic'] = split_topic_desc[1]

    else:
        pass

0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
20  Test
['']
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 

In [60]:
df.to_csv('Results_2.csv', index=False)

In [4]:
# from top2vec import Top2Vec
# model = Top2Vec(documents=df['Description'].tolist())



KeyboardInterrupt

